# Optimizando el Flujo de Trabajo en Desarrollo y DevOps con Sugar

![](https://raw.githubusercontent.com/osl-incubator/sugar/main/artwork/v1/logo.png)

## Repositorio

https://github.com/osl-incubator/sugar

## Setup

In [ ]:
# !pip install "containers-sugar==1.9.2"

## Sugar

Problemas que intenta solucionar sugar:

* facilita manejo de distintos grupos de servicios de uno o más archivos docker-compose
* facilita el uso de archivos de environment
* soporte a docker-compose v1 y v2
* soporte a casi todos los comandos de docker-compose (v2)
* definición de grupos de servicios por variables de entorno
* posibilidad de extensión de comandos

Let's start with the docker-compose.yaml:

In [1]:
%%writefile docker-compose-sugar.yaml
version: "3.9"
services:
  s1:
    image: python:3.11
    command: "python -m http.server $S1_PORT"
  s2:
    image: python:3.11
    command: "python -m http.server $S2_PORT"
  s3:
    image: python:3.11
    command: "python -m http.server $S3_PORT"

Overwriting docker-compose-sugar.yaml


NOTA: necesitamos instalar docker-compose. Para ello, instalamos el paquete compose-go que es un wrapper para docker-compose 2 escrito go.

In [ ]:
# !pip install compose-go

Problemas con el format yaml? 
Verifique tu archivo con https://www.yamllint.com/

In [2]:
%%writefile .env.sugar
S1_PORT=18001
S2_PORT=18002
S3_PORT=18003
SUGAR_GROUP=dev
SUGAR_PROJECT_NAME=pyconbo2023

Overwriting .env.sugar


In [3]:
# !docker compose config --env-file .env
!docker compose config

name: pyconbolivia2023
services:
  s1:
    command:
    - python
    - -m
    - http.server
    - "18000"
    image: python:3.11
    networks:
      default: null
  s2:
    command:
    - python
    - -m
    - http.server
    - "18001"
    image: python:3.11
    networks:
      default: null
  s3:
    command:
    - python
    - -m
    - http.server
    - "18002"
    image: python:3.11
    networks:
      default: null
networks:
  default:
    name: pyconbolivia2023_default


In [ ]:
!docker compose --env-file .env.sugar --file docker-compose-sugar.yaml config

In [ ]:
!docker compose --env-file .env.sugar --project-name osl --file docker-compose-sugar.yaml config

Para todos los comando de docker compose, vamos a necesitar pasar los mismo parámetros siempre:

`--env-file .env.sugar --project-name osl --file docker-compose-sugar.yaml`.

Problemas:

* es molestoso tener que escribir todos estos parámetros a cada vez que usamos docker compose
* podemos olvidarnos de poner uno de los parámetros
* podemos poner informaciones incorrectas

In [ ]:
!docker compose --help

* **Solución 1:** Usar *Make* o **Makim**.
* **Solución 2:** Usar **Sugar**

### Let's play with Sugar

Let's have fun!

In [4]:
%%writefile .sugar.yaml
version: 1.0
env-file: .env.sugar
compose-app: docker compose
defaults:
  group: {{ env.SUGAR_GROUP }}
  project-name: sugar-{{ env.SUGAR_PROJECT_NAME }}
groups:
  - name: prod
    compose-path: 
      - docker-compose-sugar.yaml
    env-file: .env.sugar
    services:
      default: s1,s2
      available:
        - name: s1
        - name: s2

  - name: dev
    compose-path: 
      - docker-compose-sugar.yaml
    env-file: .env.sugar
    services:
      default: s1
      available:
        - name: s1

  - name: staging
    compose-path: 
      - docker-compose-sugar.yaml
    env-file: .env.sugar
    services:
      default: s3
      available:
        - name: s3


Overwriting .sugar.yaml


In [5]:
!sugar --help

usage: sugar [-h] [--verbose] [--version] [--service-group SERVICE_GROUP]
             [--services SERVICES] [--service SERVICE] [--all]
             [--config-file CONFIG_FILE] [--options OPTIONS] [--cmd CMD]
             [plugin] [action]

sugar (or sugar) is a tool that help you to organizeand simplify your
containers' stack.

positional arguments:
  plugin                Specify the plugin/extension for the command
  action                Specify the command to be performed.

options:
  -h, --help            show this help message and exit
  --verbose             Show the command executed.
  --version             Show the version of sugar.
  --service-group SERVICE_GROUP, --group SERVICE_GROUP
                        Specify the group name of the services you want to use
  --services SERVICES   Set the services for the container call. Use comma to
                        separate the services's name
  --service SERVICE     Set the service for the container call.
  --all            

In [7]:
!cat .env.sugar

S1_PORT=18001
S2_PORT=18002
S3_PORT=18003
SUGAR_GROUP=dev
SUGAR_PROJECT_NAME=pyconbo2023


In [6]:
!sugar pull

[+] Pulling 0/0
 ⠋ s1 Pulling                                                              0.1s 
[+] Pulling 0/1
 ⠙ s1 Pulling                                                              0.2s 
[+] Pulling 0/1
 ⠹ s1 Pulling                                                              0.3s 
[+] Pulling 0/1
 ⠸ s1 Pulling                                                              0.4s 
[+] Pulling 0/1
 ⠼ s1 Pulling                                                              0.5s 
[+] Pulling 0/1
 ⠴ s1 Pulling                                                              0.6s 
[+] Pulling 0/1
 ⠦ s1 Pulling                                                              0.7s 
[+] Pulling 0/1
 ⠧ s1 Pulling                                                              0.8s 
[+] Pulling 0/1
 ⠇ s1 Pulling                                                              0.9s 
[+] Pulling 0/1
 ⠏ s1 Pulling                                                              1.0s 
[+] Pulling 0/1
 ⠋ s1 Pulling 

In [8]:
%%writefile .env.sugar
S1_PORT=18001
S2_PORT=18002
S3_PORT=18003
SUGAR_GROUP=prod
SUGAR_PROJECT_NAME=pyconbo2023

Overwriting .env.sugar


In [9]:
!sugar pull

[+] Pulling 1/0
 ✔ s1 Skipped - Image is already being pulled by s2                        0.0s 
 ⠋ s2 Pulling                                                              0.1s 
[+] Pulling 1/2
 ✔ s1 Skipped - Image is already being pulled by s2                        0.0s 
 ⠙ s2 Pulling                                                              0.2s 
[+] Pulling 1/2
 ✔ s1 Skipped - Image is already being pulled by s2                        0.0s 
 ⠹ s2 Pulling                                                              0.3s 
[+] Pulling 1/2
 ✔ s1 Skipped - Image is already being pulled by s2                        0.0s 
 ⠸ s2 Pulling                                                              0.4s 
[+] Pulling 1/2
 ✔ s1 Skipped - Image is already being pulled by s2                        0.0s 
 ⠼ s2 Pulling                                                              0.5s 
[+] Pulling 1/2
 ✔ s1 Skipped - Image is already being pulled by s2                        0.0s 
 ⠴ s2 Pulling

In [11]:
%%writefile .env.sugar
S1_PORT=18001
S2_PORT=18002
S3_PORT=18003
SUGAR_GROUP=staging
SUGAR_PROJECT_NAME=pyconbo2023

Overwriting .env.sugar


In [12]:
!sugar pull

[+] Pulling 0/0
 ⠋ s3 Pulling                                                              0.1s 
[+] Pulling 0/1
 ⠙ s3 Pulling                                                              0.2s 
[+] Pulling 0/1
 ⠹ s3 Pulling                                                              0.3s 
[+] Pulling 0/1
 ⠸ s3 Pulling                                                              0.4s 
[+] Pulling 0/1
 ⠼ s3 Pulling                                                              0.5s 
[+] Pulling 0/1
 ⠴ s3 Pulling                                                              0.6s 
[+] Pulling 0/1
 ⠦ s3 Pulling                                                              0.7s 
[+] Pulling 0/1
 ⠧ s3 Pulling                                                              0.8s 
[+] Pulling 0/1
 ⠇ s3 Pulling                                                              0.9s 
[+] Pulling 0/1
 ⠏ s3 Pulling                                                              1.0s 
[+] Pulling 0/1
 ⠋ s3 Pulling 

In [13]:
!sugar pull --group dev

[+] Pulling 0/0
 ⠋ s1 Pulling                                                              0.1s 
[+] Pulling 0/1
 ⠙ s1 Pulling                                                              0.2s 
[+] Pulling 0/1
 ⠹ s1 Pulling                                                              0.3s 
[+] Pulling 0/1
 ⠸ s1 Pulling                                                              0.4s 
[+] Pulling 0/1
 ⠼ s1 Pulling                                                              0.5s 
[+] Pulling 0/1
 ⠴ s1 Pulling                                                              0.6s 
[+] Pulling 0/1
 ⠦ s1 Pulling                                                              0.7s 
[+] Pulling 0/1
 ⠧ s1 Pulling                                                              0.8s 
[+] Pulling 0/1
 ⠇ s1 Pulling                                                              0.9s 
[+] Pulling 0/1
 ⠏ s1 Pulling                                                              1.0s 
[+] Pulling 0/1
 ⠋ s1 Pulling 